In [2]:
from __future__ import print_function
from __future__ import division
from gensim.models import Doc2Vec, Word2Vec
import lib.text_extraction as te
from functools import partial
import matplotlib.pyplot as plt
import matplotlib
from lib.parallelize import parallelize
from collections import OrderedDict
import lib.embedding as em
import lib.words as wd
import pandas as pd
import numpy as np
import operator
import codecs
import math
import json
import csv
import os
import re

matplotlib.style.use('ggplot')

Using TensorFlow backend.


## Sentencing

In [9]:

def list_dir(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [ ]:
test_dir = '../extraction/esempi_descrizioni/'

filenames = os.listdir(test_dir)
full_filenames = list_dir(test_dir)

txts = [te.extract_text(f, do_ocr=True, pages=-1) for f in full_filenames]

#txts = parallelize(te.extract_text)(full_filenames, do_ocr=True, pages=-1) #Doesn't work with do_ocr=True

In [ ]:
for i, txt in zip(filenames, txts):
    print('Documento ', i)
    print(txt[:1000])

In [ ]:
extracted_txts_dir = '../extraction/esempi_extracted/'

for fn, txt in zip(filenames, txts):
    utxt = wd.to_utf8(txt)
    with codecs.open(os.path.join(extracted_txts_dir, fn[:-3])+'txt', 'w', encoding='utf-8') as o:
        o.write(utxt)


In [ ]:
sentences_dir = '../extraction/esempi_sentences/'

sentences = [wd.sentences_doc(txt) for txt in txts]

for fn, txt in zip(filenames, sentences):
    with codecs.open(os.path.join(sentences_dir, fn[:-3])+'txt', 'w', encoding='utf-8') as o:
        o.write('\n'.join(s.replace('\n',' ').strip() for s in txt if s.strip()))

## Saving sentences

In [ ]:
sentenced_txts = [wd.sentences_doc(txt) for txt in txts]

In [ ]:
def sentence_label_csv_empty(filenames, sentenced_txts, csv_out, sep=u'\t', min_len=10):
    csv_out.write(sep.join([u'filename',u'sent_index',u'sentence',u'label']) + u'\n')
    for f, sents in zip(filenames, sentenced_txts):
        for i, sent in enumerate(sents):
            clean_sent = sent.replace(sep,u'').replace(u'\n',u' ').strip()
            if len(clean_sent)>=min_len:
                csv_out.write(sep.join([u'{}',u'{}',u'{}',u'non_rilevante']).format(f,i,clean_sent)+u'\n')
    

In [ ]:
csv_filename = '../extraction/sentence_labels.csv'
with codecs.open(csv_filename, 'w', encoding='utf-8') as csv_out:
    sentence_label_csv_empty(os.listdir(test_dir), sentenced_txts, csv_out)

In [ ]:
df = pd.read_csv(csv_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

In [ ]:
df

In [ ]:
labelled_filename = '../extraction/sentence_manual_labels.csv'
dfm = pd.read_csv(labelled_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

## Save to excel files

In [24]:
def sentences_to_excel(sentences, filename, out_dir, sep=u'\t', min_len=10):
    clean_sents = [sent.replace(sep,u'').replace(u'\n',u' ').strip() for sent in sentences]
    clean_sents = [sent for sent in clean_sents if len(sent)>=min_len]
    df = pd.DataFrame({'sentence':clean_sents,'label':['non_rilevante']*len(clean_sents)})
    df.to_excel(os.path.join(out_dir, os.path.basename(filename)[:-3]+'xlsx'), index=False, encoding='utf-8')

In [25]:
files_to_label = [f for f in list_dir('../extraction/files_to_label/') if f.endswith('.pdf')]
txts_to_label = [te.extract_text(f, do_ocr=True, pages=-1) for f in files_to_label]
sentences_to_label = [wd.sentences_doc(txt) for txt in txts_to_label]

In [26]:
for f, s in zip(files_to_label, sentences_to_label):
    sentences_to_excel(s, f, '../extraction/excels_to_label/')

,label,sentence
0,non_rilevante,Rep N 38477 Racc N 27559 COSTITUZIONE DI SOCIE...
1,non_rilevante,il quale interviene al presente atto sia in no...
2,non_rilevante,"- CELLINI MASSIMO, nato a Roma il 25 marzo 196..."
3,non_rilevante,"Detti comparenti, della cui identità personale..."
4,non_rilevante,Il primo esercizio sociale si chiude il 31 dic...
5,non_rilevante,I comparenti nominano Amministratore Unico a t...
6,non_rilevante,Viene versato il 25% del suddetto capitale all...
7,non_rilevante,"- Euro 5000,00 (cinquemila virgola zero zero) ..."
8,non_rilevante,"- Euro 2500,00 (duemilacinquecento virgola zer..."
9,non_rilevante,Intestato alla Società


## Experiments on labelled sentences

In [3]:
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

def splitted_words_lower(sentences):
    return [[w.lower() for w in wd.splitted_words_utf8(s)] for s in sentences]


In [4]:
labeled_csv_filename = '../extraction/sentence_manual_labels3.csv'

ldf = pd.read_csv(labeled_csv_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

In [5]:
ldf.groupby('label').count()

,filename,sent_index,sentence
label,,,
clausola,166,166,166
finanziario,7,7,7
gestione,90,90,90
non_rilevante,2077,2077,2077
poteri,26,26,26


In [ ]:
labels = ['poteri', 'gestione', 'finanziario', 'clausola']

sentences_rows = {l:ldf.loc[ldf['label'] == l]['sentence'] for l in labels}

splits = {l:splitted_words_lower(sentences_rows[l]) for l in labels}

In [1]:
def word_counts(sentences):
    d = dict()
    for sentence in sentences:
        for word in sentence:
            if not word in d:
                d[word] = 1
            else:
                d[word] += 1
    return d

def first_n_words(sentences, n):
    wc = word_counts(sentences)
    sorted_wc = sorted(wc.items(), key=operator.itemgetter(1))
    return list(reversed([x for x in sorted_wc[-n:]]))


In [ ]:
first_n_words(split_poteri, 50)

## Tf Idf


In [ ]:
def tf(word, sentence):
    '''Term frequency in the sentence'''
    return sum(1 for w in sentence if w == word)/len(sentence)

def idf(word, documents):
    '''Inverse document frequency'''
    D = len(documents)
    den = 1+sum(1 for sentence in documents if word in sentence)
    return math.log(D/den)

def idf_memo(documents):
    '''Defines a memoized version of idf with the given set of documents'''
    word_idf = {}
    def memoized_idf(word):
        if not word in word_idf:
            word_idf[word] = idf(word, documents)
        return word_idf[word]
    return memoized_idf

def tf_idf(word, sentence, sentences):
    '''Simple tf-idf index'''
    return tf(word, sentence)*idf(word, sentences)

def tf_idf_memo(documents):
    '''Defines a memoized version of tf-idf with the given set of documents'''
    idf_memoized = idf_memo(documents)
    def tf_idf_memoized(word, sentence):
        return tf(word, sentence)*idf_memoized(word)
    return tf_idf_memoized
    

In [ ]:
# On all sentences
split_sentences = [[w.lower() for w in wd.splitted_words_utf8(s)] for s in ldf['sentence']]
tf_idf_memoized = tf_idf_memo(split_sentences)

In [ ]:
def sorted_tfidf_words(splitted_sentences , tf_idf_memoized):
    words = set(word for sent in splitted_sentences for word in sent)
    words_mean_tfidf = [(word, np.mean([tf_idf_memoized(word, s) for s in splitted_sentences])) for word in words]
    return sorted(words_mean_tfidf, key=operator.itemgetter(1), reverse=True)

In [ ]:
sorted_words = {k:sorted_tfidf_words(v, tf_idf_memoized) for k,v in splits.items()}


In [ ]:
def print_parole_tf_idf(sorted_words, n = -1):
    print(u'{:20}\t{:20}\n'.format(u'parola',u'mean tf-idf'))
    if n < 0:
        n = len(sorted_words)
    for w, score in sorted_words[:n]:
        print(u'{:20}\t{:1.4f}'.format(w,score))

In [ ]:
for k,v in sorted_words.items():
    print('{}:'.format(k))
    print_parole_tf_idf(v, 10)
    print()

In [ ]:
most_relevant_poteri_words = OrderedDict(sorted_words['poteri']) #[:8]

In [ ]:
def score_sentence_frequency(sentence, most_relevant_words):
    '''cosine distance between frequency of words in sentence and the most relevant words tf-idf'''
    counts = OrderedDict((w, 0) for w in most_relevant_words.keys())
    for w in sentence:
        if w in counts:
            counts[w] += 1
    l = len(sentence)
    frequencies = [v/l for v in counts.values()]
    if np.linalg.norm(frequencies) == 0 or np.linalg.norm(most_relevant_words.values()) == 0:
        return 0
    return cosine_similarity(np.array(frequencies), np.array(most_relevant_words.values())) 
    
def score_sentence_tf_idf(sentence, most_relevant_words, tf_idf_memoized):
    '''cosine distance between tf-idf of words in sentence and the most relevant words tf-idf'''
    tfidfs = OrderedDict((w, 0) for w in most_relevant_words.keys())
    for w in sentence:
        tfidfs[w] = tf_idf_memoized(w, sentence)
    if np.linalg.norm(tfidfs.values()) == 0 or np.linalg.norm(most_relevant_words.values()) == 0:
        return 0
    l = min(len(tfidfs), len(most_relevant_words))
    return cosine_similarity(np.array(tfidfs.values())[:l], np.array(most_relevant_words.values())[:l])

In [ ]:
[score_sentence_tf_idf(split, OrderedDict(sorted_words['poteri']), tf_idf_memoized) for split in splits['poteri']]

In [ ]:
score_sentence_tf_idf(split_sentences[1432], OrderedDict(sorted_words['poteri']), tf_idf_memoized)

## Embeddings test

In [ ]:
txts_tokenized = [wd.tokenize_doc(txt) for txt in txts]

In [ ]:
reduced_dictionary_filename = 'first_5000_words.json'
with open(reduced_dictionary_filename) as f:
    reduced_dictionary = set(json.load(f))

gensim_model_filename = 'models/gensim_model_5000.d2v'
gensim_model = Doc2Vec.load(gensim_model_filename)

In [ ]:
embeddings = [em.embed_document(gensim_model, txt, reduced_dictionary) for txt in txts_tokenized]

In [ ]:
sv = partial(em.sentence_vector, model=gensim_model, permitted_words=reduced_dictionary)

In [ ]:
p1 = u'All\'organo amministrativo sono conferiti i più ampi poteri, sia per la gestione ordinaria che straordinaria della Società'
p2 = u'ARTICOLO 19 - POTERI DELL\'ORGANO AMMINISTRATIVO 191 L\'organo amministrativo ha tutti i poteri di ordinaria e straordinaria amministrazione'
cosine_similarity(sv(sentence=p1.lower()),sv(sentence=p2.lower())) 

In [ ]:
mean_cosines_poteri = [np.mean([cosine_similarity(sv(sentence=split_poteri[i]), sv(sentence=split_poteri[j]))
                               for i in range(len(split_poteri)) if i != j])
                                   for j in range(len(split_poteri))]

In [ ]:
mean_cosines_poteri

## Word2Vec


In [ ]:
sentences_tokenized = [sent for doc in txts_tokenized for sent in doc] #i should reduce the dictionary also

In [ ]:
w2v = Word2Vec(sentences_tokenized, size=100, window=5, min_count=5, workers=4)

In [ ]:
w2v.wv['costitutivo']

In [ ]:
cosine_similarity(w2v.wv['atto'], w2v.wv['costitutivo'])

In [ ]:
w2v.most_similar('poteri')

In [ ]:
w2v.most_similar(positive=['atto','costitutivo'])